In [1]:
# Run this cell to see if things work
import sys

import torch
from torch.nn import Parameter
import torch.nn as nn
import torch.nn.functional as F

import syft as sy
hook = sy.TorchHook(torch)

torch.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

In [2]:
# Basic tools of private
x = torch.tensor([1,2,3,4,5])
y = x + x
print(y)

tensor([ 2,  4,  6,  8, 10])


#  ```x_ptr = tensor.send(bob)```to send
#  ```x_ptr.get()``` to get back from the virtual worker

In [3]:
#Sending Tensors to Bob's Machine
bob = sy.VirtualWorker(hook, id="bob")

In [4]:
x = torch.tensor([1,2,3,4,5])
y = torch.tensor([1,1,1,1,1])

In [5]:
x_ptr = x.send(bob)
y_ptr = y.send(bob)

In [6]:
x_ptr

(Wrapper)>[PointerTensor | me:87433398567 -> bob:65681133796]

In [7]:
bob._objects

{65681133796: tensor([1, 2, 3, 4, 5]), 53941314337: tensor([1, 1, 1, 1, 1])}

In [8]:
z = x_ptr + x_ptr

In [9]:
z

(Wrapper)>[PointerTensor | me:28357754044 -> bob:67869923737]

In [10]:
bob._objects

{65681133796: tensor([1, 2, 3, 4, 5]),
 53941314337: tensor([1, 1, 1, 1, 1]),
 67869923737: tensor([ 2,  4,  6,  8, 10])}

In [11]:
###############
x_ptr

(Wrapper)>[PointerTensor | me:87433398567 -> bob:65681133796]

In [12]:
x_ptr.location

<VirtualWorker id:bob #objects:3>

In [13]:
#
x_ptr.id_at_location

65681133796

In [14]:
x_ptr.id

87433398567

In [15]:
x_ptr.location
#############

<VirtualWorker id:bob #objects:3>

In [16]:
me = sy.local_worker
me

<VirtualWorker id:me #objects:0>

In [17]:
me == x_ptr.owner

True

# ``` get ```

In [18]:
x_ptr.get()
y_ptr.get()
z.get()

tensor([ 2,  4,  6,  8, 10])

In [19]:
bob._objects# empty

{}

# Section1.2 Using Tensor Pointers

In [20]:
x = torch.tensor([1,2,3,4,5]).send(bob)
y = torch.tensor([1,1,1,1,1]).send(bob)

In [21]:
z = x + y

In [22]:
z

(Wrapper)>[PointerTensor | me:25124912023 -> bob:58232781164]

在幕后，发生了一件非常强大的事情。不是 x 和 y 在本地计算加法，而是将命令序列化并发送给 Bob，Bob 执行计算，创建张量 z

In [23]:
bob._objects

{96650805046: tensor([1, 2, 3, 4, 5]),
 25231547036: tensor([1, 1, 1, 1, 1]),
 58232781164: tensor([2, 3, 4, 5, 6])}

# Torch Functions

In [24]:
x

(Wrapper)>[PointerTensor | me:81351500441 -> bob:96650805046]

In [25]:
y

(Wrapper)>[PointerTensor | me:48186066495 -> bob:25231547036]

In [26]:
z = torch.add(x,y)
z

(Wrapper)>[PointerTensor | me:18871915809 -> bob:38399233196]

In [27]:
bob._objects

{96650805046: tensor([1, 2, 3, 4, 5]),
 25231547036: tensor([1, 1, 1, 1, 1]),
 58232781164: tensor([2, 3, 4, 5, 6]),
 38399233196: tensor([2, 3, 4, 5, 6])}

In [28]:
z.get()

tensor([2, 3, 4, 5, 6])

In [29]:
bob._objects

{96650805046: tensor([1, 2, 3, 4, 5]),
 25231547036: tensor([1, 1, 1, 1, 1]),
 58232781164: tensor([2, 3, 4, 5, 6])}

In [30]:
z.get()

AttributeError: 'Tensor' object has no attribute 'child'

In [31]:
bob._objects

{96650805046: tensor([1, 2, 3, 4, 5]),
 25231547036: tensor([1, 1, 1, 1, 1]),
 58232781164: tensor([2, 3, 4, 5, 6])}

# Variables (including backpropagation!)

In [32]:
x = torch.tensor([1,2,3,4,5.], requires_grad=True).send(bob)
y = torch.tensor([1,1,1,1,1.], requires_grad=True).send(bob)

In [33]:
x.grad

(Wrapper)>[PointerTensor | me:82066742397 -> bob:85816897080]::grad

In [34]:
(x + y).sum()

(Wrapper)>[PointerTensor | me:34229591003 -> bob:90006590941]

In [35]:
z = (x + y).sum()
z

(Wrapper)>[PointerTensor | me:2944480381 -> bob:13532594689]

In [36]:
bob._objects

{96650805046: tensor([1, 2, 3, 4, 5]),
 25231547036: tensor([1, 1, 1, 1, 1]),
 58232781164: tensor([2, 3, 4, 5, 6]),
 85816897080: tensor([1., 2., 3., 4., 5.], requires_grad=True),
 385816462: tensor([1., 1., 1., 1., 1.], requires_grad=True),
 90006590941: tensor(20., grad_fn=<SumBackward0>),
 13532594689: tensor(20., grad_fn=<SumBackward0>)}

In [37]:
z.backward()

(Wrapper)>[PointerTensor | me:61439840902 -> bob:4342242731]

In [38]:
x = x.get()
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [39]:
x.grad

tensor([1., 1., 1., 1., 1.])

#  Part X - Secure Training and Evaluation on MNIST